In [5]:
import os
import pandas as pd
from itertools import product
from glob import glob
from gzip import BadGzipFile


In [6]:
# Define the date range
start_year = 2017
end_year = 2017

# Specify the desired months and days
desired_months = [9] # list(range(1, 13))  # Adjust this for desired months (e.g., [1, 2, 3] for Jan, Feb, Mar)
desired_days = [6] # list(range(1, 32))    # Adjust this for desired days (e.g., [1, 15, 30])

# List of acronyms
acronyms = ["ATHA", "FSIM", "FSMI", "GILL", "PINA", "RANK", "SNKQ"]

# Directory where the files are located (adjust as needed)
data_dir = './carisma_data/'  # Assuming files are in a directory named 'carisma_data'

# Dictionary to hold the DataFrames for each acronym
dfs_dict = {acronym: [] for acronym in acronyms}


In [7]:
# Loop through each combination of year, month, day, and acronym using itertools.product
for year, month, day, acronym in product(range(start_year, end_year + 1), desired_months, desired_days, acronyms):
    folder_name = f'{year}-{month:02d}-{day:02d}'
    file_path = os.path.join(data_dir, folder_name, f'{year}{month:02d}{day:02d}{acronym}.F02.gz.txt')
    
    print(f"Trying to access: {file_path}")
    
    if os.path.exists(file_path):
        try:
            # Attempt to read it as a plain text file
            data = pd.read_csv(file_path, header=None)
            print(f"Successfully read the file as a plain text file: {file_path}")
            
            data_list = data[0].tolist()
            reformatted_data = []
            i = 1
            while i < len(data_list) - 2:
                timestamp = data_list[i]
                if str(timestamp).split()[0].isdigit():
                    timestamp = timestamp.replace(' .', '')
                    timestamp = timestamp.replace(' x', '')
                    timestamp_dt = pd.to_datetime(timestamp, format='%Y%m%d%H%M%S')
                    values1 = list(map(float, data_list[i + 1].split()))
                    values2 = list(map(float, data_list[i + 2].split()))
                    averaged_values = [(v1 + v2) / 2 for v1, v2 in zip(values1, values2)]
                    averaged_row = [timestamp_dt] + averaged_values
                    reformatted_data.append(averaged_row)
                    i += 3
                else:
                    i += 1
            df_averaged = pd.DataFrame(reformatted_data, columns=["Timestamp", "x", "y", "z"])
            dfs_dict[acronym].append(df_averaged)
            
        except Exception as e:
            print(f"Error reading the file as plain text: {e}")
            try:
                data = pd.read_csv(file_path, header=None, compression='gzip')
                print(f"Successfully read the file as a gzipped file: {file_path}")
                
            except BadGzipFile:
                print(f"File {file_path} is not a gzipped file.")
    else:
        print(f"File not found: {file_path}")

Trying to access: ./carisma_data/2017-09-06/20170906ATHA.F02.gz.txt


Successfully read the file as a plain text file: ./carisma_data/2017-09-06/20170906ATHA.F02.gz.txt
Trying to access: ./carisma_data/2017-09-06/20170906FSIM.F02.gz.txt
Successfully read the file as a plain text file: ./carisma_data/2017-09-06/20170906FSIM.F02.gz.txt
Trying to access: ./carisma_data/2017-09-06/20170906FSMI.F02.gz.txt
Successfully read the file as a plain text file: ./carisma_data/2017-09-06/20170906FSMI.F02.gz.txt
Trying to access: ./carisma_data/2017-09-06/20170906GILL.F02.gz.txt
Successfully read the file as a plain text file: ./carisma_data/2017-09-06/20170906GILL.F02.gz.txt
Trying to access: ./carisma_data/2017-09-06/20170906PINA.F02.gz.txt
Successfully read the file as a plain text file: ./carisma_data/2017-09-06/20170906PINA.F02.gz.txt
Trying to access: ./carisma_data/2017-09-06/20170906RANK.F02.gz.txt
Successfully read the file as a plain text file: ./carisma_data/2017-09-06/20170906RANK.F02.gz.txt
Trying to access: ./carisma_data/2017-09-06/20170906SNKQ.F02.gz.tx

In [8]:
# Display the DataFrames
for acronym, dfs in dfs_dict.items():
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        print(f"Data for {acronym}:")
        display(combined_df.head())

Data for ATHA:


,Timestamp,x,y,z
0,2017-09-06 00:00:00,14055.5470,-470.0045,55431.8855
1,2017-09-06 00:00:01,14055.4055,-470.1185,55431.8460
2,2017-09-06 00:00:02,14055.2780,-470.2250,55431.8135
3,2017-09-06 00:00:03,14055.2055,-470.3045,55431.8100
4,2017-09-06 00:00:04,14054.9890,-470.2915,55431.7670


Data for FSIM:


,Timestamp,x,y,z
0,2017-09-06 00:00:00,11501.7880,2842.6510,57300.4465
1,2017-09-06 00:00:01,11501.8995,2842.7975,57300.2530
2,2017-09-06 00:00:02,11502.0075,2842.9855,57300.1030
3,2017-09-06 00:00:03,11502.1295,2843.1810,57299.9280
4,2017-09-06 00:00:04,11502.3615,2843.4045,57299.7595


Data for FSMI:


...

,Timestamp,x,y,z
0,2017-09-06 00:00:00,15477.3900,-11.3725,55285.1190
1,2017-09-06 00:00:01,15477.3235,-11.4590,55285.1060
2,2017-09-06 00:00:02,15477.2690,-11.5115,55285.0595
3,2017-09-06 00:00:03,15477.2210,-11.5570,55285.0565
4,2017-09-06 00:00:04,15477.1920,-11.6410,55285.0280


Data for RANK:


,Timestamp,x,y,z
0,2017-09-06 00:00:00,6305.5255,417.3755,58871.8875
1,2017-09-06 00:00:01,6305.5835,417.4120,58871.9220
2,2017-09-06 00:00:02,6305.5790,417.3795,58871.9280
3,2017-09-06 00:00:03,6305.4885,417.3735,58871.9345
4,2017-09-06 00:00:04,6305.4615,417.4115,58871.9655


Data for SNKQ:


,Timestamp,x,y,z
0,2017-09-06 00:00:00,11414.843,27.5810,56240.0705
1,2017-09-06 00:00:01,11416.292,29.6420,56242.0860
2,2017-09-06 00:00:02,11415.116,26.1825,56239.7260
3,2017-09-06 00:00:03,11416.275,29.3055,56242.6360
4,2017-09-06 00:00:04,11415.569,29.3960,56242.4240


In [ ]:
# ######## WRITE RESULTING FILES INTO A FOLDER NAMED 'output_data' WHICH USER WILL HAVE TO CREATE

# output_directory = "./output_data/"  # Adjust this to the desired directory for the output .dat files

# # Ensure the output directory exists
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# # Construct the date range string for the filename (Output will look like: ATHA_data_20170906_to_20170906.dat)
# date_range_str = f"{start_year}{desired_months[0]:02d}{desired_days[0]:02d}_to_{end_year}{desired_months[-1]:02d}{desired_days[-1]:02d}"

# # Save the DataFrames to .dat files
# for acronym, df in dfs_dict.items():
#     if df is not None:
#         output_file_path = os.path.join(output_directory, f"{acronym}_data.dat")
#         df.to_csv(output_file_path, index=False, sep="\t")  # Using tab separator for .dat files
#         print(f"Saved data for {acronym} to: {output_file_path}")